In [3]:
import pandas as pd
from sagemaker.sklearn.estimator import SKLearn
import time
import boto3
import json
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.session import Session
from io import StringIO
import time

sess = boto3.Session()
sm = sess.client('sagemaker')
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import boto3

client = boto3.client('rds')
response = client.describe_db_instances(
    DBInstanceIdentifier='sa2-db'
)
print(response['DBInstances'][0]['Endpoint']['Address'])
hostname=response['DBInstances'][0]['Endpoint']['Address']
connection = pg.connect(user = 'postgres',
password = 'sa2dbroot',
host = str(hostname),
port = '5432',
database = 'yash_sa_schema')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
data = pd.read_sql_query('SELECT * FROM yash_sa_schema.sa_file_ingst_log ORDER BY sa_file_ingst_log.upld_ts DESC LIMIT 1',con=connection)
print(data['published_file_path'].iloc[0])
#print(data['data_src_nm'].iloc[0])
print(data['published_file_name'].iloc[0])
print(data['model'].iloc[0])
Type=data['file_upld_typ'].iloc[0]
model=data['model'].iloc[0]
DatasetPath=data['published_file_path'].iloc[0]
filename=data['published_file_name'].iloc[0]
#DatasetPath=data['DatasetPath'].iloc[0]


---------------!

In [ ]:
import sklearn
import numpy as np
from sklearn.externals import joblib
import os
import sagemaker
from sagemaker.sklearn.model import SKLearnModel
new_role = sagemaker.get_execution_role()
#change the model_data parameter below to the location of tar.gz containing joblib/pickle
model = SKLearnModel(model_data=DatasetPath,entry_point='entry_point.py',source_dir='./DataFolder',role = new_role)

#model = SKLearnModel(model_data="s3://sagemaker-us-east-1-754307369999/test_joblib_deployment/model.tar.gz",entry_point='model_entry_point_v1.py',source_dir='./DataFolder',role = new_role)
# entry point script in ./DataFolder has version v1 for joblib and v2 for pickle seperately. ./DatFolder also contains requirements.txt for dependencies.

#model = SKLearnModel(model_data="s3://sagemaker-us-east-1-754307369999/test_joblib_deployment/model3.tar.gz",entry_point='model_entry_point_v2.py',source_dir='./DataFolder',role = new_role)
# similar model objects need to be created depending upon the framework used to train the model. the entry_point script loads the model from joblib/pickle and returns the model object.

predictor = model.deploy(instance_type="ml.m5.large", initial_instance_count=1)